# Análise comparativa de resultados

## 1. Preparação dos dados

### 1.1 Configurações iniciais

In [91]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from pathlib import Path
# Modelling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, classification_report
from sklearn.preprocessing import (
    OneHotEncoder, StandardScaler, RobustScaler, OrdinalEncoder, LabelEncoder
)
import warnings

### 1.2 Obtendo os dados e classificando variáveis

In [92]:
#obtendo conjunto de dados
data_path = Path('../data/raw/data.csv')
dict_path = Path('../data/external/dicionario.csv')

In [93]:
# Ler conjunto de dados
df = (
    pd.read_csv(data_path)
)
# Ler dicionário de dados
df_dict = pd.read_csv(dict_path)
df_dict

,variavel,significado,tipo,subtipo,resposta
0,gender,Gênero do aluno,Qualitativa,Nominal,False
1,NationalITy,Nacionalidade do aluno,Qualitativa,Nominal,False
2,PlaceofBirth,Local de nascimento do aluno,Qualitativa,Nominal,False
3,StageID,Nível de escolaridade a que o aluno pertence,Qualitativa,Ordinal,False
4,GradeID,A série em que o aluno está matriculado,Qualitativa,Ordinal,False
5,SectionID,Sala de aula à qual o aluno pertence,Qualitativa,Nominal,False
6,Topic,Tópico do curso,Qualitativa,Nominal,False
7,Semester,Semestre do ano letivo,Qualitativa,Ordinal,False
8,Relation,Progenitor responsável pelo aluno,Qualitativa,Nominal,False
9,raisedhands,Número de vezes que o aluno levantou a mão,Quantitativa,Discreta,False


In [94]:
# Separar variáveis

variaveis_interesse = df_dict.query('resposta == False').variavel.to_list()
variavel_resposta = df_dict.query('resposta == True').variavel.to_list()


df_X = df[variaveis_interesse]
df_y = df[variavel_resposta]

# Separar variáveis de interesse por tipo
nominal_columns = df_dict.query('variavel in @variaveis_interesse and subtipo == "Nominal"').variavel.to_list()
continuos_columns = df_dict.query('variavel in @variaveis_interesse and subtipo == "Discreta"').variavel.to_list()

### 1.3 Tratando dados 

In [95]:
# criando pipelines para tratar os dados

nominal_preprocessor = Pipeline(steps=[
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes ()
    ('encoding', OneHotEncoder(sparse=False, handle_unknown='ignore')), # Codificação de variáveis
    # Seleção de variáveis
     ('normalization', StandardScaler()) # Normalização de variáveis
])
continuous_preprocessor = Pipeline(steps=[
    # Tratamento de dados discrepantes
    ('missing', KNNImputer(n_neighbors=5)), # Tratamento de dados faltantes
    # Seleção de variáveis
    ('normalization', RobustScaler()) # Normalização
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuos', continuous_preprocessor, continuos_columns)
])

preprocessor

ColumnTransformer(transformers=[('nominal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False)),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['gender', 'NationalITy', 'PlaceofBirth',
                                  'SectionID', 'Topic', 'Relation',
                                  'ParentAnsweringSurvey',
                                  'ParentschoolSatisfaction']),
                                ('continuos',
                                 Pipeline(steps=[('missing', KNNImputer()),
                                                 ('normalization',
                                                  RobustScaler())]),
                                 ['raisedhands', 'VisITedResources',
                                  'AnnouncementsView', 'Discussion'])])

In [96]:
preprocessor.fit(df_X)

X = preprocessor.transform(df_X)

X

C:\Users\letic\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[-0.7574764 ,  0.7574764 , -0.1382327 , ..., -0.765625  ,
        -0.70454545, -0.38      ],
       [-0.7574764 ,  0.7574764 , -0.1382327 , ..., -0.703125  ,
        -0.68181818, -0.28      ],
       [-0.7574764 ,  0.7574764 , -0.1382327 , ..., -0.90625   ,
        -0.75      , -0.18      ],
       ...,
       [ 1.32017315, -1.32017315, -0.1382327 , ...,  0.140625  ,
        -0.18181818, -0.2       ],
       [ 1.32017315, -1.32017315, -0.1382327 , ..., -0.75      ,
        -0.43181818,  0.36      ],
       [ 1.32017315, -1.32017315, -0.1382327 , ..., -0.796875  ,
        -0.22727273,  0.46      ]])

In [97]:
y = df_y['Class'].replace({'L': 0, 'M': 1, 'H':2}).to_numpy().ravel()

y

array([1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 2, 1, 0, 0, 2, 1, 1, 1, 1, 2, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 2, 2, 1, 0, 0, 1, 2, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 0, 0,
       1, 2, 2, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 2, 0, 0, 0, 1, 2, 0, 2, 0,
       0, 0, 0, 2, 2, 2, 0, 2, 2, 1, 1, 1, 1, 2, 0, 0, 1, 0, 1, 2, 1, 1,
       2, 1, 0, 0, 0, 0, 1, 2, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 2, 2, 2, 1, 2, 1, 0, 0, 1, 2, 0, 1, 2, 1, 1, 2, 2, 1, 2, 0,
       1, 2, 1, 1, 0, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 0, 0, 1, 0,
       2, 1, 2, 1, 2, 0, 2, 1, 0, 2, 1, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 2,
       0, 1, 2, 2, 1, 1, 0, 2, 1, 1, 1, 1, 2, 1, 2, 0, 0, 0, 1, 1, 2, 1,
       1, 1, 1, 2, 2, 1, 0, 0, 2, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 2, 2, 1,
       0, 1, 2, 1, 2, 1, 0, 1, 2, 0, 1, 0, 2, 2, 2, 1, 1, 0, 0, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1,
       2, 2, 1, 1, 0, 0, 2, 2, 1, 1, 2, 2, 1, 1, 0,

In [98]:
# separaando dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((384, 55), (96, 55))

## 2. Escolha do modelo

### 2.1 Metodologia

Iremos análisar quatro modelos, que serão testados utilizando um método de validação cruzada por holdout, os modelos que serão testados serão:

* Linear Regression 
* K-Nearest-Neighbors
* Random Forest Classifier
* Logistic Regression

### - Função de avaliação para fornecer as métricas após o treinamento do modelo

In [99]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [100]:
models = {
    "Linear Regression": LinearRegression(),
    "K-Nearest-Neighbors": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Desempenho do modelo para conjunto de treinamento')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Desempenho do modelo para conjunto de teste')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Desempenho do modelo para conjunto de treinamento
- Root Mean Squared Error: 0.4568
- Mean Absolute Error: 0.3689
- R2 Score: 0.6359
----------------------------------
Desempenho do modelo para conjunto de teste
- Root Mean Squared Error: 0.4806
- Mean Absolute Error: 0.3924
- R2 Score: 0.5364


K-Nearest-Neighbors
Desempenho do modelo para conjunto de treinamento
- Root Mean Squared Error: 0.6312
- Mean Absolute Error: 0.3255
- R2 Score: 0.3047
----------------------------------
Desempenho do modelo para conjunto de teste
- Root Mean Squared Error: 0.7430
- Mean Absolute Error: 0.4688
- R2 Score: -0.1080


Random Forest
Desempenho do modelo para conjunto de treinamento
- Root Mean Squared Error: 0.0000
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000
----------------------------------
Desempenho do modelo para conjunto de teste
- Root Mean Squared Error: 0.5103
- Mean Absolute Error: 0.2604
- R2 Score: 0.4774


Logistic Regression
Desempenho do modelo para conjunto d

In [101]:
pd.DataFrame(list(zip(model_list, r2_list)), 
             columns=[
                 'Modelo', 'R2_Score'
                 ]).sort_values(
                     by=[
                         "R2_Score"
                         ],
                         ascending=False)

,Modelo,R2_Score
0,Linear Regression,0.536363
2,Random Forest,0.477352
3,Logistic Regression,0.226481
1,K-Nearest-Neighbors,-0.108014


### Linear Regression


In [102]:
model = LinearRegression()
model.fit(X_test,y_test)

LinearRegression()

In [103]:
y_pred = model.predict(X_test)

In [104]:
pred_df=pd.DataFrame({'Valor Atual':y_test,'Valor Previsto':y_pred,'Diferença':y_test-y_pred})
pred_df

,Valor Atual,Valor Previsto,Diferença
0,1,1.062012,-0.062012
1,0,-0.027344,0.027344
2,2,1.757324,0.242676
3,2,1.802734,0.197266
4,2,2.248535,-0.248535
...,...,...,...
91,1,0.752930,0.247070
92,1,1.486816,-0.486816
93,1,0.538086,0.461914
94,0,0.392578,-0.392578


### Random Forest Classifier

In [105]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [106]:
y_pred = clf.predict(X_test)

In [107]:
class_report = classification_report(y_test,y_pred)
print("Classification Report:\n", class_report)

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.81      0.81        26
           1       0.74      0.71      0.72        48
           2       0.62      0.68      0.65        22

    accuracy                           0.73        96
   macro avg       0.72      0.73      0.73        96
weighted avg       0.73      0.73      0.73        96



### Logistic Regression

In [108]:
logistic_regression = LogisticRegression()

logistic_regression.fit(X_train,y_train)

y_pred = logistic_regression.predict(X_test)

In [109]:
class_report = classification_report(y_test,y_pred)
print("Classification Report:\n", class_report)

Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.73      0.69        26
           1       0.63      0.56      0.59        48
           2       0.54      0.59      0.57        22

    accuracy                           0.61        96
   macro avg       0.61      0.63      0.62        96
weighted avg       0.62      0.61      0.61        96



### Resultados

Por meio das métricas análisadas (precision, recall, f1-score, f2-score), foi possível observar que o **Random Forest** obteve maior número de predições corretas, sendo, nesta análise comparativa, o modelo mais adequado. Enquanto o **K-Nearest-Neighbors** obteve o menor número de predições corretas, sendo o modelo menos indicato para nossas predições.